# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.offsetbox import (OffsetImage,AnnotationBbox)
from matplotlib import transforms
import matplotlib.animation as animation
import time as time

sns.set_theme()

# 2. Data Understanding
Trên thực tế việc khám phá dữ liệu đã được nhóm thực hiện một phần ở notebook Data Preprocessing. Ở notebook này, nhóm chủ yếu tìm hiểu sự phân bố của dữ liệu ở các cột
# 2.1. Tổng quan kích thước, kiểu dữ liệu

In [ ]:
df = pd.read_csv('data/kill_match_stats_v2.csv')
df.head()

In [ ]:
df.shape

Dữ liệu có 8,253,143 dòng vaf 9 cột

Mỗi dòng mô tả một kill diễn ra trong trận đấu, gồm:
- `killed_by`: nguyên nhân gây kill
- `kx` và `ky`: tọa của killer trên bản đồ 
- `time`: thời điểm kill xảy ra (tính từ đầu trận đấu)
- `vx` và `vy`: tọa của victim trên bản đồ
- `dis`: khoảng cách giữa killer và victim
- `phase`: phase diễn ra kill
- `type`: phân loại nguyên nhân gây kill

In [ ]:
df.info()

Dữ liệu của các cột:
- `killed_by` và `type`: nominal
- `kx`, `ky`, `vx`, `vy`, `dis`: numerical
- `time` và `phase`: 2 cột này mang ý nghĩa thời điểm, nên nhóm xếp 2 cột này vào nhóm dữ liệu `thời gian`

In [ ]:
df = pd.read_csv('data/kill_match_stats_v2.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

## 2.2. Phân bố của dữ liệu trong từng cột
### 2.2.1. Các cột tọa độ

In [ ]:
fig, axes = plt.subplots(ncols = 2, nrows = 2, figsize=(12, 8), sharex = 'col', sharey = 'row')
fig.subplots_adjust(wspace = 0.1)

axes[0][0].hist(df['kx'], bins = 100, edgecolor = 'C0')
axes[0][1].hist(df['ky'], bins = 100, edgecolor = 'C0')
axes[1][0].hist(df['vx'], bins = 100, color = 'C1', edgecolor = 'C1')
axes[1][1].hist(df['vy'], bins = 100, color = 'C1', edgecolor = 'C1')

axes[0][0].set_title('kx')
axes[0][1].set_title('ky')
axes[1][0].set_title('vx')
axes[1][1].set_title('vy');

### 2.2.2. Cột dis

In [ ]:
fig = plt.figure(figsize=(12, 6))

sns.histplot(df['dis'], bins = 1000)

### 2.2.3. Cột `time` và `phase`

In [ ]:
fig, axes = plt.subplots(ncols = 2, figsize = (15, 5))

axes[0].hist(df['time'], bins = 1000, edgecolor = 'C0')

data = df.groupby(by = ['phase']).count().reset_index()[['phase', 'type']]
sns.barplot(data = data, x = 'phase', y = 'type', ax = axes[1], color = 'C1')

axes[0].set_title('time')
axes[1].set_title('phase')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

y, _, _ = plt.hist(df['time'], bins = 1000, edgecolor = 'C0')

phase_rect = [(0, 720), (721, 1060), (1061, 1300), (1301, 1480), (1481, 1650), (1651, 1760), (1761, 1880), (1881, 1970), (1971, 2150)]

for rect in phase_rect:
    ax.add_patch(patches.Rectangle((rect[0], 0), rect[1] - rect[0], y.max(), color = 'C1', alpha = 0.2))

### 2.2.4. Cột `killed_by` và `type`

In [ ]:
data = df['killed_by'].value_counts().reset_index()

fig, ax = plt.subplots(figsize=(8, 40))
ax = sns.barplot(data = data, x = 'killed_by', y = 'index', color = 'C0')

ax.set_xlim(0, data['killed_by'].max() * 1.3)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xlabel('')
ax.set_ylabel('')
ax.grid(visible = False)
ax.set_title('Killed_by count')

#annotate the bars
for i, v in enumerate(data['killed_by']):
    ax.text(v + 10000, i, f'{v:,}', va = 'center', fontsize = 11)

#use asset item as tick labels
for i, v in enumerate(data['index']):  
    img = plt.imread('asset/item/' + v +'.png', format = 'png')

    imagebox = OffsetImage(img, zoom = 0.07)
    imagebox.image.axes = ax

    ab = AnnotationBbox(imagebox, (0, 0),
                        xybox=(-10000, i),
                        xycoords=("data"),
                        box_alignment = (1, 0.5),
                        bboxprops = {'edgecolor': 'none'})

    ax.add_artist(ab)


In [ ]:
data = df['type'].value_counts().reset_index()

fig, ax = plt.subplots(figsize=(8, 8))
ax = sns.barplot(data = data, x = 'type', y = 'index', color = 'C0')

ax.set_xlim(0, data['type'].max() * 1.3)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_xlabel('')
ax.set_ylabel('')
ax.grid(visible = False)
ax.set_title('Type count')

#annotate the bars
for i, v in enumerate(data['type']):
    ax.text(v + 100000, i, f'{v:,}', va = 'center', fontsize = 11)

#use icon as tick labels
for i, v in enumerate(data['index']):  
    img = plt.imread('asset/type_item/' + v +'.png')

    imagebox = OffsetImage(img, zoom = 0.05)
    imagebox.image.axes = ax

    ab = AnnotationBbox(imagebox, (0, 0),
                        xybox=(-100000, i),
                        xycoords=("data"),
                        box_alignment = (1, 0.5),
                        bboxprops = {'edgecolor': 'none'})

    ax.add_artist(ab)

## 2.3. Phân bố dữ liệu giữa các cột với nhau
### 2.3.1. Đếm `killed_by` theo từng `type`

In [ ]:
datas = [
    df[df['type'] == 'AR']['killed_by'].value_counts().reset_index(),
    df[(df['type'] == 'SMG') | (df['type'] == 'Shotgun')]['killed_by'].value_counts().reset_index(),
    df[(df['type'] == 'DMR') | (df['type'] == 'SR')]['killed_by'].value_counts().reset_index(),
    df[(df['type'] == 'Handgun')]['killed_by'].value_counts().reset_index(),
    df[(df['type'] == 'Melee')]['killed_by'].value_counts().reset_index(),
    df[(df['type'] == 'Throwable') | (df['type'] == 'Zone') | (df['type'] == 'Self') | (df['type'] == 'LMG')]['killed_by'].value_counts().reset_index()
]

titles = ['AR', 'SMG & Shotgun', 'DMR & SR', 'Handgun', 'Melee', 'Throwable & Zone & Self & LMG']

In [ ]:
fig, axes = plt.subplots(ncols = 2, nrows = 3, figsize = (15, 15))
fig.subplots_adjust(hspace = 0.4)

for i, ax in enumerate(axes.flat):
    sns.barplot(data = datas[i], x = 'index', y = 'killed_by', ax = ax, color = f'C{i}')
    ax.set_title(titles[i])

    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticklabels([])
    ax.grid(visible = False)
    
#annotate the bars
for ax in axes.flat:
    max_h = max([v.get_height() for v in ax.patches])
    ax.set_ylim(0, max_h * 1.15)
    for j, v in enumerate(ax.patches):
        ax.text(v.get_x() + v.get_width() / 2, v.get_height() + max_h * 0.03, f'{int(v.get_height()):,}', ha = 'center', fontsize = 11)

#use asset item as tick labels
for ax in axes.flat:
    max_h = max([v.get_height() for v in ax.patches])
    for i, v in enumerate(ax.get_xticklabels()):
        img = plt.imread('asset/item/' + v.get_text() +'.png', format = 'png')

        imagebox = OffsetImage(img, zoom = 0.07)
        imagebox.image.axes = ax

        ab = AnnotationBbox(imagebox, (0, 0),
                            xybox=(i, -max_h * 0.15),
                            xycoords=("data"),
                            box_alignment = (0.5, 0.5),
                            bboxprops = {'edgecolor': 'none'})

        ax.add_artist(ab)
    ax.set_xticklabels([])

#save the figure
for ax in axes.flat:
    extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

### 2.3.2. Đếm `type` theo `time`

In [ ]:
data = df[['type', 'time', 'killed_by']].groupby(by = ['type', 'time']).count().reset_index()
data.columns = ['type', 'time', 'count']

In [ ]:
data['type'].value_counts()[0:5].index

In [ ]:
plt.figure(figsize = (16, 5))
sns.lineplot(data = data[data['type'].isin(data['type'].value_counts()[0:6].index)], x = 'time', y = 'count', hue = 'type', palette = 'tab10')
plt.title('Top 6 upper of Type count by time')

In [ ]:
plt.figure(figsize = (16, 5))
sns.lineplot(data = data[data['type'].isin(data['type'].value_counts()[6:].index)], x = 'time', y = 'count', hue = 'type', palette = 'tab10')
plt.title('Top 6 lower of Type count by time')

### 2.3.3. Trung bình khoảng cách theo thời gian

In [ ]:
df[['time', 'dis']].groupby(by = ['time']).mean().plot(figsize = (16, 5), title = 'Average distance by time (included Bluezone)')

In [ ]:
df[df['killed_by'] != 'BlueZone'][['time', 'dis']].groupby(by = ['time']).mean().plot(figsize = (16, 5), title = 'Average distance by time (excluded Bluezone)')

In [ ]:
fig, axes = plt.subplots(nrows = 2, figsize = (16, 11))
fig.subplots_adjust(hspace = 0.4)

data = df[df['type'].isin(df['type'].value_counts()[0:6].index)][['time', 'type', 'dis']].groupby(by = ['time', 'type']).mean().reset_index()
sns.scatterplot(data = data, x = 'time', y = 'dis', hue = 'type', palette = 'Set1_r', ax = axes[0], s = 2, edgecolor = None)
axes[0].set_title('Top 6 upper of Type average distance by time')

data = df[df['type'].isin(df['type'].value_counts()[6:].index)][['time', 'type', 'dis']].groupby(by = ['time', 'type']).mean().reset_index()
sns.scatterplot(data = data, x = 'time', y = 'dis', hue = 'type', palette = 'Set1', ax = axes[1], s = 2, edgecolor = None)
axes[1].set_title('Top 6 lower of Type average distance by time')

### 2.3.4. Phân bố dữ liệu theo tọa độ (2 chiều)

In [ ]:
fig, axes = plt.subplots(ncols = 2, nrows = 2, figsize=(12, 8), sharex = 'col', sharey = 'row')
fig.subplots_adjust(wspace = 0.1)

axes[0][0].hist(df['kx'], bins = 100, edgecolor = 'C0')
axes[0][1].hist(df['ky'], bins = 100, edgecolor = 'C0')
axes[1][0].hist(df['vx'], bins = 100, color = 'C1', edgecolor = 'C1')
axes[1][1].hist(df['vy'], bins = 100, color = 'C1', edgecolor = 'C1')

axes[0][0].set_title('kx')
axes[0][1].set_title('ky')
axes[1][0].set_title('vx')
axes[1][1].set_title('vy');

### 2.3.5. Phân bố dữ liệu theo tọa độ và thời gian

In [ ]:
img = plt.imread('asset/map/ERANGEL.png')

In [ ]:
%matplotlib inline

def ax_prunt(ax):
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlim(0, number_of_bin)
    ax.set_ylim(0, number_of_bin)
    ax.invert_yaxis()

def init():
    sns.heatmap(np.zeros((number_of_bin, number_of_bin)), vmin = 0, vmax = 8, cmap = 'nipy_spectral', cbar = False)
    
    ax_prunt(ax)
def animate(i):
    data = df[df['time'].isin(range(i, i + 50))][['vx', 'vy']]
    data = np.log(plt.hist2d(data['vy'], data['vx'], bins = number_of_bin, range = [[0, 8000], [0, 8000]])[0] + 1)
    sns.heatmap(data, cmap = 'nipy_spectral', vmin = 0, vmax = 8, cbar = False)

    ax.set_title(f'{i}s - {i + 50}s')
    ax_prunt(ax)
    print(i)

fig, ax = plt.subplots(figsize = (10, 8))
number_of_bin = 1600
scale = number_of_bin / 1600 * 1640

plt.imshow(img[::-1, :, 0], extent = [0, scale, 0, scale], alpha = 0.35, zorder = 3, cmap = 'gray')
ax_prunt(ax)
sns.heatmap(np.zeros((number_of_bin, number_of_bin)), vmin = 0, vmax = 8, cmap = 'nipy_spectral')

anim = animation.FuncAnimation(fig, func = animate, init_func = init, interval = 20, frames = range(1300, df['time'].max() - 40, 20), repeat = False)

start_time = time.time()

writer = animation.FFMpegWriter(fps=10)
anim.save('test3.mp4', writer=writer)

print(f'{time.time() - start_time:.2f} sec for saving video')  

In [ ]:
df.shape

In [ ]:
df.info()